### Подготовка выборки ткани

In [4]:
import pandas as pd
from random import sample, randint, seed
from IPython.display import clear_output
seed(10)

In [8]:
# Загрузка последовательности
def update_sequence(postfix):
    fin = open(r"create_models_project/hg19_human/hg19_" + postfix + ".fasta", 'r')
    seq = "".join(str(x)[:-1] for x in fin.readlines())
    seq = seq[len("assembly") + seq.find("assembly") : ]
    fin.close()
    return seq

In [3]:
data_narrowPeaks = pd.read_csv(r"tissue/ENCFF503TXI.bed", sep='\t', header=None)
data_narrowPeaks

,0,1,2,3,4,5,6,7,8,9
0,chr10,100027597,100027921,Peak_19633,27,.,6.50848,12.08685,9.86127,165
1,chr10,100028184,100028626,Peak_12137,141,.,23.86444,77.19582,74.60011,193
2,chr10,100099770,100100609,Peak_18537,32,.,6.13069,15.09537,12.84007,195
3,chr10,100109284,100109722,Peak_15949,55,.,9.75909,27.82287,25.47054,147
4,chr10,100174436,100175093,Peak_17525,39,.,8.67798,18.60225,16.31486,388
...,...,...,...,...,...,...,...,...,...,...
23414,chrY,2802584,2805392,Peak_9758,235,.,35.43507,130.53738,127.72445,355
23415,chrY,2870860,2872612,Peak_9267,256,.,37.96615,142.86922,140.00575,389
23416,chrY,7140748,7141890,Peak_9351,253,.,37.60456,141.09486,138.24051,745
23417,chrY,7141950,7144189,Peak_12237,139,.,23.50285,75.62300,73.03380,710


In [4]:
data_narrowPeaks = data_narrowPeaks.sort_values(0, ignore_index=True)
data_narrowPeaks

,0,1,2,3,4,5,6,7,8,9
0,chr1,1822575,1824102,Peak_12436,131,.,19.41992,71.23882,68.66872,545
1,chr1,207924844,207926438,Peak_4170,490,.,53.47513,275.74731,272.19550,903
2,chr1,207994013,207997897,Peak_2464,569,.,52.61066,321.10211,317.20605,2613
3,chr1,207999386,207999978,Peak_15292,63,.,9.13400,32.71488,30.33136,204
4,chr1,208040922,208042513,Peak_8272,304,.,36.86970,169.70337,166.72618,1079
...,...,...,...,...,...,...,...,...,...,...
23414,chrY,14774205,14775740,Peak_5023,451,.,57.21048,253.73027,250.32109,380
23415,chrY,14773460,14774118,Peak_12340,135,.,22.26755,73.33641,70.75821,395
23416,chrY,7141950,7144189,Peak_12237,139,.,23.50285,75.62300,73.03380,710
23417,chrY,21703296,21703992,Peak_20518,24,.,5.82382,10.52341,8.31709,450


In [5]:
len(data_narrowPeaks)

23419

In [6]:
len(data_narrowPeaks[data_narrowPeaks[0] == "chr1"])

2344

In [7]:
indexes_learn = sorted(sample(list(data_narrowPeaks.index), 20000))

indexes_test = []
for i in range(len(data_narrowPeaks)):
    if i not in indexes_learn:
        indexes_test.append(i)
indexes_test = sorted(sample(indexes_test, 3000))

#тестируем
for i in range(len(indexes_learn)-1):
    if indexes_learn[i] + 1 != indexes_learn[i+1]:
        print("True")
        break
for i in indexes_test:
    if i in indexes_learn:
        print("False")
        break
print(len(indexes_learn) == 20000)
print(len(indexes_test) == 3000)

True
True
True


In [8]:
with open(r"H3K4me3_learn.fasta", 'w') as fout:
    postfix = "chr1"
    seq = ""
    seq = update_sequence(postfix)
    for i in indexes_learn:
        if postfix != data_narrowPeaks[0][i]:
            postfix = data_narrowPeaks[0][i]
            seq = update_sequence(postfix)
        
        middle = (int(data_narrowPeaks[1][i]) + int(data_narrowPeaks[2][i])) // 2
        left = max(middle - 500, 0)
        right = min(middle + 500, len(seq))
        
        fout.write(">" + postfix + ":" + str(left) + "-" + str(right) + "\n")
        fout.write(seq[left:right] + "\n")

In [9]:
with open(r"H3K4me3_test.fasta", 'w') as fout:
    postfix = "chr1"
    seq = ""
    seq = update_sequence(postfix)
    for i in indexes_test:
        if postfix != data_narrowPeaks[0][i]:
            postfix = data_narrowPeaks[0][i]
            seq = update_sequence(postfix)
        
        middle = (int(data_narrowPeaks[1][i]) + int(data_narrowPeaks[2][i])) // 2
        left = max(middle - 500, 0)
        right = min(middle + 500, len(seq))
        
        fout.write(">" + postfix + ":" + str(left) + "-" + str(right) + "\n")
        fout.write(seq[left:right] + "\n")

In [2]:
# Тестируем
with open(r"create_models_project/gen_samples_data/H3K4me3/train_data/H3K4me3_learn.fasta", 'r') as test:
    l = test.readlines()
    print(len(l) == 40000)
    b = True
    for i in range(1, 40000, 2):
        if len(l[i]) != 1001:
            b = False
    print(b)

with open(r"create_models_project/gen_samples_data/H3K4me3/train_data/H3K4me3_test.fasta", 'r') as test:
    l = test.readlines()
    print(len(l) == 6000)
    b = True
    for i in range(1, 6000, 2):
        if len(l[i]) != 1001:
            b = False
    print(b)

True
True
True
True


### Подготовка данных для эмуляции ввода

In [21]:
with open(r"input.fasta", 'w') as fout:
    ind = randint(0, len(data_narrowPeaks) - 1)
    
    postfix = data_narrowPeaks[0][ind]
    seq = update_sequence(postfix)
    
    middle = (int(data_narrowPeaks[1][ind]) + int(data_narrowPeaks[2][ind])) // 2
    left = max(middle - 500, 0)
    right = min(middle + 500, len(seq))
    
    fout.write(">" + postfix + ":" + str(left) + "-" + str(right) + "\n")
    fout.write(seq[left:right] + "\n")

### Форматирование hg19

In [12]:
# Форматирование файла сборки генома hg19
with open(r"hg19_human/hg19.fa", 'r') as file:
    fin = open(r"hg19_human/hg19.fasta", 'w')
    for line in file:
        if line[0] == '>':
            fin.write(line)
        else:
            fin.write(line.upper())
    fin.close()

### Подготовка random sample

In [12]:
random_sample = pd.read_csv(r"random_sample_intersected.bed", sep='\t', header=None)
random_sample = random_sample.sort_values(0, ignore_index=True)
random_sample

,0,1,2
0,chr1,5928670,5929058
1,chr1,207098789,207100320
2,chr1,17958130,17959677
3,chr1,2533134,2534681
4,chr1,2251224,2251540
...,...,...,...
1016205,chrY,25344066,25345442
1016206,chrY,13476223,13477599
1016207,chrY,21364917,21365320
1016208,chrY,9212240,9215639


In [13]:
indexes_random_learn = sorted(sample(list(random_sample.index), 60000))

indexes_random_test = []
while len(indexes_random_test) < 9000:
    ind = randint(0, len(random_sample) - 1)
    if ind not in indexes_random_test and ind not in indexes_random_learn:
        indexes_random_test.append(ind)
            
indexes_random_learn = sorted(indexes_random_learn)
indexes_random_test = sorted(indexes_random_test)
            
#тестируем
for i in range(len(indexes_random_learn) - 1):
    if indexes_random_learn[i] + 1 != indexes_random_learn[i+1]:
        print("True")
        break
for i in range(len(indexes_random_test)-1):
    if indexes_random_test[i] + 1 != indexes_random_test[i+1]:
        print("True")
        break
for i in indexes_random_learn:
    if i in indexes_random_test:
        print("False")
        break
print(len(indexes_random_learn) == 60000)
print(len(indexes_random_test) == 9000)

True
True
True
True


In [14]:
with open(r"H3K4me3_random_learn.fasta", 'w') as fout:
    postfix = "chr1"
    seq = ""
    seq = update_sequence(postfix)
    for i in indexes_random_learn:
        if postfix != random_sample[0][i]:
            postfix = random_sample[0][i]
            seq = update_sequence(postfix)
        
        middle = (int(random_sample[1][i]) + int(random_sample[2][i])) // 2
        left = max(middle - 500, 0)
        right = min(middle + 500, len(seq))
        
        fout.write(">" + postfix + ":" + str(left) + "-" + str(right) + "\n")
        fout.write(seq[left:right] + "\n")

In [15]:
with open(r"H3K4me3_random_test.fasta", 'w') as fout:
    postfix = "chr1"
    seq = ""
    seq = update_sequence(postfix)
    for i in indexes_random_test:
        if postfix != random_sample[0][i]:
            postfix = random_sample[0][i]
            seq = update_sequence(postfix)
        
        middle = (int(random_sample[1][i]) + int(random_sample[2][i])) // 2
        left = max(middle - 500, 0)
        right = min(middle + 500, len(seq))
        
        fout.write(">" + postfix + ":" + str(left) + "-" + str(right) + "\n")
        fout.write(seq[left:right] + "\n")

In [3]:
# Тестируем
with open(r"create_models_project/gen_samples_data/H3K4me3/train_data/H3K4me3_random_learn.fasta", 'r') as test:
    l = test.readlines()
    print(len(l) == 120000)
    b = True
    for i in range(1, 120000, 2):
        if len(l[i]) != 1001:
            b = False
    print(b)

with open(r"create_models_project/gen_samples_data/H3K4me3/train_data/H3K4me3_random_test.fasta", 'r') as test:
    l = test.readlines()
    print(len(l) == 18000)
    b = True
    for i in range(1, 18000, 2):
        if len(l[i]) != 1001:
            b = False
    print(b)

True
True
True
True


In [11]:
postfix = "chr1"
seq = ""
seq = update_sequence(postfix)
seq[2501]

'N'